# CMU 10-405/605 Machine Learning with Large Datasets
## **Welcome to Recitation 2**
This notebook will walk you through a basic Spark transformations and actions. We will also process large Wikipedia dump to find interesting insights about your favorite celebrities.

#### ** Part 1: Basic Spark Transformations and Actions **
Refer to the Spark guide: https://spark.apache.org/docs/2.4.4/rdd-programming-guide.html

In [3]:
# create a sample list
my_list = [i for i in range(1,10000000)]

# parallelize the data
rdd_0 = sc.parallelize(my_list)

# add value 4 to each number
rdd_1 = rdd_0.map(lambda x : x + 4)

# print the value on the console
rdd_1.take(10)

# What will the above line print?
# What happens to rdd_0?

Out[8]: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [4]:
# Make tuples
rdd_2 = rdd_0.map(lambda x: (x, x+1))

rdd_2.take(10)

Out[9]: [(1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (10, 11)]

In [5]:
# Include decimal numbers 
rdd_3 = rdd_0.flatMap(lambda x: (x-0.5, x, x+0.5))

rdd_3.take(10)

Out[10]: [0.5, 1, 1.5, 1.5, 2, 2.5, 2.5, 3, 3.5, 3.5]

In [6]:
# Remove the duplicates
rdd_4 = rdd_3.distinct().cache()

rdd_4.take(10)

Out[13]: [5000192.5,
 5000200,
 5000200.5,
 5000208,
 5000208.5,
 5000216,
 5000216.5,
 5000224,
 5000224.5,
 5000232]

In [7]:
# Sort the values
rdd_5 = rdd_4.map(lambda x: (x, 1)).sortByKey().map(lambda x: x[0])

rdd_5.take(10)

Out[14]: [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

In [8]:
# Get all the duplicates using groupByKey
rdd_6 = rdd_3.map(lambda x: (x, 1)).groupByKey().filter(lambda x: len(x[1]) > 1)
rdd_6.take(10)

Out[15]: [(8.5, <pyspark.resultiterable.ResultIterable at 0x7ff206caeda0>),
 (16.5, <pyspark.resultiterable.ResultIterable at 0x7ff206cae198>),
 (24.5, <pyspark.resultiterable.ResultIterable at 0x7ff206caee10>),
 (32.5, <pyspark.resultiterable.ResultIterable at 0x7ff206caebe0>),
 (40.5, <pyspark.resultiterable.ResultIterable at 0x7ff206caef60>),
 (48.5, <pyspark.resultiterable.ResultIterable at 0x7ff206cae6d8>),
 (56.5, <pyspark.resultiterable.ResultIterable at 0x7ff206caed68>),
 (64.5, <pyspark.resultiterable.ResultIterable at 0x7ff206cae588>),
 (72.5, <pyspark.resultiterable.ResultIterable at 0x7ff206caee80>),
 (80.5, <pyspark.resultiterable.ResultIterable at 0x7ff206caeb00>)]

In [9]:
# Get all the duplicates using reducebyKey
rdd_7 = rdd_3.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 1)
rdd_7.take(10)

Out[16]: [(1249280.5, 2),
 (1249288.5, 2),
 (1249296.5, 2),
 (1249304.5, 2),
 (1249312.5, 2),
 (1249320.5, 2),
 (1249328.5, 2),
 (1249336.5, 2),
 (1249344.5, 2),
 (1249352.5, 2)]

#### ** Part 2: Exploratory Data Analysis on Wikipedia dump **

In [11]:
# Load the Wikipedia dataset
wikiRDD = sc.wholeTextFiles("/FileStore/tables/text/*/*")

wikiRDD.take(10)

# What's the difference between textFile and wholeTextFiles?

Out[1]: [('dbfs:/FileStore/tables/text/AA/wiki_00',
 '<doc id="12" url="https://en.wikipedia.org/wiki?curid=12" title="Anarchism">\nAnarchism\n\nAnarchism is an anti-authoritarian political and social philosophy that rejects hierarchies as unjust and advocates their replacement with self-managed, self-governed societies based on voluntary, cooperative institutions. These institutions are often described as stateless societies, although several authors have defined them more specifically as distinct institutions based on non-hierarchical or free associations. Anarchism\'s central disagreement with other ideologies is that it holds the state to be undesirable, unnecessary, and harmful.\n\nAnarchism is usually placed on the far-left of the political spectrum, and much of its economics and legal philosophy reflect anti-authoritarian interpretations of communism, collectivism, syndicalism, mutualism, or participatory economics. As anarchism does not offer a fixed body of doctrine from a single particular worldview, many anarchist types and traditions exist and varieties of anarchy diverge widely. Anarchist schools of thought can differ fundamentally, supporting anything from extreme individualism to complete collectivism. Strains of anarchism have often been divided into the categories of social and individualist anarchism, or similar dual classifications.\n\nThe etymological origin of the word anarchism is from the ancient Greek word "anarkhia", meaning "without a ruler", composed of the prefix "a-" (i.e. "without") and the word "arkhos" (i.e. "leader" or "ruler"). The suffix -ism denotes the ideological current that favours anarchy. The word anarchism appears in English from 1642 as "anarchisme" and the word "anarchy" from 1539. Various factions within the French Revolution labelled their opponents as anarchists, although few such accused shared many views with later anarchists. Many revolutionaries of the 19th century such as William Godwin (1756–1836) and Wilhelm Weitling (1808–1871) would contribute to the anarchist doctrines of the next generation, but they did not use the word anarchist or anarchism in describing themselves or their beliefs.\n\nThe first political philosopher to call himself an anarchist () was Pierre-Joseph Proudhon (1809–1865), marking the formal birth of anarchism in the mid-19th century. Since the 1890s and beginning in France, the term libertarianism has often been used as a synonym for anarchism and its use as a synonym is still common outside the United States. On the other hand, some use libertarianism to refer to individualistic free-market philosophy only, referring to free-market anarchism as libertarian anarchism.\n\nWhile opposition to the state is central to anarchist thought, defining anarchism is not an easy task as there is a lot of talk among scholars and anarchists on the matter and various currents perceive anarchism slightly differently. Hence, it might be true to say that anarchism is a cluster of political philosophies opposing authority and hierarchical organization (including the state, capitalism, nationalism and all associated institutions) in the conduct of all human relations in favour of a society based on voluntary association, on freedom and on decentralisation, but this definition has the same shortcomings as the definition based on etymology (which is simply a negation of a ruler), or based on anti-statism (anarchism is much more than that) or even the anti-authoritarian (which is an "a posteriori" conclusion). Nonetheless, major elements of the definition of anarchism include the following:\n\nDuring the prehistoric era of mankind, an established authority did not exist. It was after the creation of towns and cities that institutions of authority were established and anarchistic ideas espoused as a reaction. Most notable precursors to anarchism in the ancient world were in China and Greece. In China, philosophical anarchism was delineated by Taoist philosophers Zhuangzi and

In [12]:
# Get all the pages
pages = wikiRDD.flatMap(lambda x: x[1].split('</doc>')).cache()

pages.count()

Out[2]: 5571559

Extract the title and content of the article and return a RDD containing tuples of (title, actual content)

In [14]:
def get_title_and_content(content):
    """
      This method generates a tuple of (title, actual content) from a Wiki dump of each page
    """
    # Remove any leading or lagging space if present 
    content = content.strip()
    title, actual_content = '', ''
    try:
        if(content != ''):
            # Split the content on the basis of new line
            arr = content.split("\n",2)
            # Second line is the title
            title = arr[1]
            # Rest is the actual content
            actual_content = arr[2]
    except:
        title, actual_content = '', ''
    return (title, actual_content)

In [15]:
# Extract the title and actual content
pages = pages.map(get_title_and_content)

pages.count()

Out[4]: 5571559

Filter out all the articles of celebrities from the dataset

In [17]:
import re
def check_if_person(content):
    """ 
      This method checks if a page is of a celebrity or not.
    """
    # Checking in first 150 characters
    content = content[:150]
    # Checking for format like 12 August 1993
    list1 = re.findall(r"[\d]{1,2} [ADFJMNOS]\w* [\d]{4}", content)
    # Checking for format like August 12, 1993
    list2 = re.findall(r"[ADFJMNOS]\w* [\d]{1,2}[,] [\d]{4}", content)
    if(len(list1)>0 or len(list2)>0):
        return True
    return False



In [18]:
# Get all the celebrities pages
celebrities = pages.filter(lambda x : check_if_person(x[1]))
celebrities.count()

Out[6]: 1282990

Get the names of the celebrities who are linked with Carnegie Mellon

In [22]:
# Get the names of the celebrities who are linked with Carnegie Mellon 
celebrities.filter(lambda x: "Carnegie Mellon" in x[1]).map(lambda x: x[0]).count()

Out[7]: 910

Get the names of the celebrities who are linked with Carnegie Mellon and have won a Turing Award

In [27]:
# Get the names of the celebrities who are linked with Carnegie Mellon and have won a Turing Award
celebrities.filter(lambda x: "Carnegie Mellon" in x[1] and "Turing" in x[1]).map(lambda x: x[0]).collect()

Out[17]: ['Alan Kay',
 'Herbert A. Simon',
 'Kevin Warwick',
 'Dana Scott',
 'Allen Newell',
 'Michael O. Rabin',
 'Alan Perlis',
 'Robert W. Floyd',
 'Edward Feigenbaum',
 'Raj Reddy',
 'Manuel Blum',
 'Geoffrey Hinton',
 'Per Brinch Hansen',
 'Lenore Blum',
 'Leslie Valiant',
 'Zoubin Ghahramani',
 'Joseph F. Traub',
 'Michael Loren Mauldin',
 'Richard Lipton',
 'Avrim Blum',
 'Edmund M. Clarke',
 'Madhavi Vuppalapati',
 'Maja Pantić']

In [28]:
# Get the names of the celebrities who are linked with Stanford and have won a Turing Award
celebrities.filter(lambda x: "Stanford" in x[1] and "Turing" in x[1]).map(lambda x: x[0]).collect()

Out[18]: ['Alan Kay',
 'Douglas Engelbart',
 'Donald Knuth',
 'Edsger W. Dijkstra',
 'Noam Chomsky',
 'Niklaus Wirth',
 'Ray Kurzweil',
 'Vint Cerf',
 'Robert Tarjan',
 'Ron Rivest',
 'Robin Milner',
 'Leslie Lamport',
 'Daniel Pearl',
 'Kenneth E. Iverson',
 'Allen Newell',
 'John Hopcroft',
 'Andrew Yao',
 'Robert W. Floyd',
 'Edward Feigenbaum',
 'Raj Reddy',
 'John McCarthy (computer scientist)',
 'Amir Pnueli',
 'H. L. A. Hart',
 'Danny Hillis',
 'Martin Hellman',
 'Whitfield Diffie',
 'John L. Hennessy',
 'Louis Essen',
 'Geoffrey K. Pullum',
 'Barbara Liskov',
 'W. M. Gorman',
 'Leslie Valiant',
 'Robin Gandy',
 'Joseph F. Traub',
 'Frances E. Allen',
 'Kenneth Colby',
 'Arthur Levenson',
 'Ashok K. Chandra',
 'Jack Andraka',
 'Robert W. Doran']